# RGB channel image maker

Code that allows you to create manipulate image RGB channels and see how they affect your plot, on-the-fly.

In [1]:
import EQ_tools as eq
from ipywidgets import interactive, VBox, HBox, Layout, widgets, fixed
import matplotlib.pyplot as plt
import numpy as np
from scipy import interpolate
import os
import sys
import xarray as xr
import matplotlib
import cartopy.feature as cfeature
import warnings
warnings.filterwarnings('ignore')

# specific tools (which can be found here ../Hub_tools/)
sys.path.append(os.path.dirname(os.getcwd()) + '/Hub_Tools/')
import image_tools as img

In [2]:
# image reduction settings: resample the image every grid_factor points
reduce_image = False
grid_factor = 5

# subset image: cut a relevant section out of an image. subset_extents [lon1,lon2,lat1,lat2] describes the section.
subset_image = True
subset_extents = [18.25, 20.75, 57.75, 59.25]

# image truncation settings
truncate_image = True
min_percentile = 5
max_percentile = 95

# image normalisation settings
unhitch = True

# image histogram settings
histogram_image = True
histogram_channels = 512

# image plotting settings: e.g. fontsize (fsz)
fsz = 10

In [3]:
vardir = '/home/jovyan/work/products/S3A_OL_1_EFR____20190809T092510_20190809T092810_20190810T145535_0179_048_036_1980_MAR_O_NT_002.SEN3'
# read lat/lon
ds1 = xr.open_dataset(os.path.join(vardir, 'geo_coordinates.nc'))
raster_lat = ds1.latitude.data
raster_lon = ds1.longitude.data
ds1.close()

if subset_image:
    i1, i2, j1, j2 = img.subset_image(raster_lat, raster_lon, subset_extents)
    raster_lat = raster_lat[i1:i2,j1:j2]
    raster_lon = raster_lon[i1:i2,j1:j2]
    subset_extents = [np.nanmin(raster_lon), np.nanmax(raster_lon),\
                      np.nanmin(raster_lat), np.nanmax(raster_lat)]

# open data
num_channels = 11

if 'EFR' in vardir:
    radiometry_type = 'Oa%s_radiance'
else:
    radiometry_type = 'Oa%s_reflectance'

for rad_channel_number in range(1, num_channels+1):
    rad_channel = radiometry_type % (str(rad_channel_number).zfill(2))
    rad_file = os.path.join(vardir, rad_channel + '.nc') 
    rad_fid = xr.open_dataset(rad_file)
    if subset_image:
        exec("Ch%s = rad_fid.%s.data[i1:i2,j1:j2]" % (str(rad_channel_number).zfill(2),rad_channel))
    else:
        exec("Ch%s = rad_fid.%s.data" % (str(rad_channel_number).zfill(2),rad_channel))
    rad_fid.close()

if 'EFR' in vardir:
    red = np.log10(1.0 + 0.01 * Ch01 + 0.09 * Ch02 + 0.35 * Ch03 + 0.04 * Ch04 + 0.01 * Ch05 + 0.59 * Ch06 + 0.85 * Ch07 + 0.12 * Ch08 + 0.07 * Ch09 + 0.04 * Ch10)
    green = np.log10(1.0 + 0.26 * Ch03 + 0.21 * Ch04 + 0.50 * Ch05 + Ch06 + 0.38 * Ch07 + 0.04 * Ch08 + 0.03 * Ch09 + 0.02 * Ch10)
    blue = np.log10(1.0 + 0.07 * Ch01 + 0.28 * Ch02 + 1.77 * Ch03 + 0.47 * Ch04 + 0.16 * Ch05)

In [4]:
if reduce_image:
    red = img.reduce_image(red, grid_factor=grid_factor)
    green = img.reduce_image(green, grid_factor=grid_factor)
    blue = img.reduce_image(blue, grid_factor=grid_factor)

In [5]:
%matplotlib inline

# build the widgets
all_widgets = eq.make_RGB_widgets(truncate_image, min_percentile, max_percentile,\
        unhitch, histogram_image, histogram_channels)

# build the plot command
runCMD = eq.build_RGB_plot_command(all_widgets)

# run the plot command
exec(runCMD)
box_layout = Layout(display='flex', flex_flow='row', justify_content='space-between', align_items='center')

# display the plot

widget_box = VBox([iplot.children[0],iplot.children[1], \
                   iplot.children[2], iplot.children[3], \
                   iplot.children[4], iplot.children[5], \
                   iplot.children[6], iplot.children[7]])
plot_box = VBox([iplot.children[-1]])
display(HBox([widget_box, plot_box]),box_layout)

Layout(align_items='center', display='flex', flex_flow='row', justify_content='space-between')